In [1]:
using Revise
using LinearAlgebra
using SparseArrays
using Pkg
using DelayDiffEq 
using Plots
using BenchmarkTools
using LaTeXStrings

In [3]:
using OpenQuantumSystems

┌ Info: Precompiling OpenQuantumSystems [90e503ea-487f-4e8e-81fe-8dcc9db25ac5]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [4]:
D(op1::Array, op2::Array) = abs(norm(op1 - op2))
D(x1::StateVector, x2::StateVector) = norm(x2 - x1)
D(op1::AbstractOperator, op2::AbstractOperator) = abs(tracedistance_nh(dense(op1), dense(op2)))
D(op1::AbstractSuperOperator, op2::AbstractSuperOperator) = abs(tracedistance_nh(dense(op1), dense(op2)))

Nvib = 3
mols = [
        Molecule([Mode(omega=200., hr_factor=0.01)], Nvib, [0., 12500.]),
    Molecule([Mode(omega=200., hr_factor=0.01)], Nvib, [0., 12700.])
    ]

aggCore = AggregateCore(mols)
for mol_i in 2:aggCore.molCount
    aggCore.coupling[mol_i, mol_i+1] = 10
    aggCore.coupling[mol_i+1, mol_i] = 10
end
agg = setupAggregate(aggCore; groundEnergy = true, vib_basis = :ground_excited)
agg.tools.bSize

27

In [11]:
tspan = get_tspan(0., 0.0001, 10)
tspan_fs = tspan_cm_to_fs(tspan)
W0, rho0, W0_bath = ultrafast_laser_excitation(10., [0., 0.7, 0.3], agg)
elLen = aggCore.molCount+1
tspan_fs[end]

0.5308837458876146

In [12]:
maxtol = 1e-6

1.0e-6

In [13]:
@time _, rho_int_t_ = Evolution_sI_exact(W0, tspan, agg)
println("")

  0.050212 seconds (41.04 k allocations: 8.185 MiB, 43.36% gc time)



In [14]:
@time _, rho_0_int_t_ = QME_sI_ansatz_const_int(
    W0,
    tspan,
    agg;
    reltol=maxtol,
    abstol=maxtol,
    int_reltol=maxtol,
    int_abstol=maxtol,
    alg = DelayDiffEq.MethodOfSteps(DelayDiffEq.Tsit5()),
)
println("")

  0.400235 seconds (1.94 M allocations: 161.778 MiB, 4.69% gc time)



In [15]:
W_0_bath_t = []
for i=1:length(tspan)
    push!(W_0_bath_t, W0_bath.data)
end

In [21]:
for be in [:none, :interaction_picture, :shroedinger_picture]
    for ba in [:population, :population_coherences]
        @time _, rho_1_int_t_, W_1_bath_t = QME_sI_iterative(
            W0,
            rho_0_int_t_,
            W_0_bath_t,
            tspan,
            agg;
            bath_evolution=be, 
            bath_ansatz=ba, 
            normalize=false,
            reltol = maxtol,
            abstol = maxtol,
            int_reltol = maxtol,
            int_abstol = maxtol,
            W_1_rtol = maxtol,
            W_1_atol = maxtol,
            K_rtol = maxtol,
            K_atol = maxtol,
            alg = DelayDiffEq.MethodOfSteps(DelayDiffEq.Tsit5()),
        )
    end
end
println("")

  6.894252 seconds (45.45 M allocations: 2.624 GiB, 6.26% gc time)
  6.272083 seconds (45.45 M allocations: 2.624 GiB, 6.38% gc time)
  7.042361 seconds (45.51 M allocations: 2.693 GiB, 5.92% gc time)
  6.902728 seconds (45.51 M allocations: 2.693 GiB, 5.84% gc time)
  6.239244 seconds (45.48 M allocations: 2.659 GiB, 6.21% gc time)
  6.356759 seconds (45.48 M allocations: 2.659 GiB, 5.98% gc time)



In [22]:
for normalize in [false, true]
    @time _, rho_1_int_t_, W_1_bath_t = QME_sI_iterative(
        W0,
        rho_0_int_t_,
        W_0_bath_t,
        tspan,
        agg;
        bath_evolution=:none, 
        bath_ansatz=:population, 
        normalize=normalize,
        reltol = maxtol,
        abstol = maxtol,
        int_reltol = maxtol,
        int_abstol = maxtol,
        W_1_rtol = maxtol,
        W_1_atol = maxtol,
        K_rtol = maxtol,
        K_atol = maxtol,
        alg = DelayDiffEq.MethodOfSteps(DelayDiffEq.Tsit5()),
    )
end
println("")

  5.701188 seconds (45.45 M allocations: 2.624 GiB, 6.88% gc time)
  5.702467 seconds (45.50 M allocations: 2.625 GiB, 6.83% gc time)

